
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src=https://raw.githubusercontent.com/aestaire/ml_workshop/refs/heads/main/files/images/hands-on.png>
</div>

# Validación del modelo Challenger

Este notebook realiza tareas de validación en el modelo candidato __Challenger__.

Pasa por algunos pasos para validar el modelo antes de etiquetarlo (asignándole el alias) como `Challenger`.

Cuando las organizaciones comienzan a implementar procesos de MLOps, deberían considerar tener un "humano en el circuito" para realizar análisis visuales y validar los modelos antes de promoverlos. A medida que se familiarizan con el proceso, pueden considerar automatizar los pasos en un __Workflow__. El beneficio de la automatización es asegurar que estas comprobaciones de validación se realicen sistemáticamente antes de que los nuevos modelos se integren en los pipelines de inferencia o se desplieguen para el servicio en tiempo real. Por supuesto, las organizaciones pueden optar por mantener un "humano en el circuito" en cualquier parte del proceso y establecer el grado de automatización que se adapte a sus necesidades empresariales.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/mlops/mlops-uc-end2end-4-v2.png?raw=true" width="1200">

*Nota: en una configuración típica de MLOps, esto se ejecutaría como parte de un trabajo automatizado para validar un nuevo modelo. Ejecutaremos esta sencilla demostración como un notebook interactivo.*

<!-- Recopilar datos de uso (vista). Elimínelo para desactivar la recopilación o desactive el rastreador durante la instalación. Consulte el README para más detalles.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=1444828305810485&notebook=%2F01-mlops-quickstart%2F04_challenger_validation&demo_name=mlops-end2end&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fmlops-end2end%2F01-mlops-quickstart%2F04_challenger_validation&version=1&user_hash=f7ea13a45c991650d8df810431c3e0e2b12887e9ed7e206ee8fb6209bdb2ae82">

### Se ha creado un cluster para esta demostración
Para ejecutar esta demostración, simplemente selecciona el cluster `dbdemos-mlops-end2end` en el menú desplegable ([abrir configuración del clúster](https://e2-demo-field-eng.cloud.databricks.com/#setting/clusters/1013-181446-g00pu2bj/configuration)). <br />
*Nota: Si el cluster fue eliminado después de 30 días, puedes recrearlo con `dbdemos.create_cluster('mlops-end2end')` o reinstalar la demo: `dbdemos.install('mlops-end2end')`*

Último ambiente testado:

mlflow==3.3.0


## Comprobaciones Generales de Validación

<!--img style="float: right" src="https://github.com/QuentinAmbard/databricks-demo/raw/main/retail/resources/images/churn-mlflow-webhook-1.png" width=600 -->

En el contexto de MLOps, existen más pruebas que simplemente la precisión de un modelo. Para garantizar la estabilidad de nuestro sistema de ML y el cumplimiento de cualquier requisito normativo, someteremos cada modelo añadido al registro a una serie de comprobaciones de validación. Estas incluyen, pero no se limitan a:
<br>
* __Documentación del modelo__
* __Inferencia sobre datos de producción__
* __Pruebas Champion-Challenger para asegurar que los KPIs de negocio sean aceptables__

En este notebook, exploramos algunos enfoques para realizar estas pruebas y cómo podemos añadir metadatos a nuestros modelos etiquetando si han pasado una prueba determinada.

Esta parte suele ser específica para tu línea de negocio y requisitos de calidad.

Para cada prueba, agregaremos información usando etiquetas para saber qué se ha validado en el modelo. También podemos añadir comentarios a un modelo si es necesario.

In [0]:
%pip install --quiet mlflow --upgrade


%restart_python

In [0]:
%run ../_resources/00-setup

In [0]:
from mlflow.store.artifact.models_artifact_repo import ModelsArtifactRepository


requirements_path = ModelsArtifactRepository(f"models:/{catalog}.{db}.mlops_churn@Challenger").download_artifacts(artifact_path="requirements.txt") # download model from remote registry

In [0]:
%pip install --quiet -r $requirements_path


%restart_python

In [0]:
%run ../_resources/00-setup

## Obtener información del modelo

Obtendremos la información del modelo __Challenger__ desde Unity Catalog.

In [0]:
# We are interested in validating the Challenger model
model_alias = "Challenger"
model_name = f"{catalog}.{db}.mlops_churn"

client = MlflowClient()
model_details = client.get_model_version_by_alias(model_name, model_alias)
model_version = int(model_details.version)

print(f"Validating {model_alias} model for {model_name} on model version {model_version}")

## Comprobaciones del modelo

#### Verificación de la descripción

¿El científico de datos proporcionó una descripción del modelo que está siendo enviado?

In [0]:
# If there's no description or an insufficient number of characters, tag accordingly
if not model_details.description:
  has_description = False
  print("Please add model description")
elif not len(model_details.description) > 20:
  has_description = False
  print("Please add detailed model description (40 char min).")
else:
  has_description = True

print(f'Model {model_name} version {model_details.version} has description: {has_description}')
client.set_model_version_tag(name=model_name, version=str(model_details.version), key="has_description", value=has_description)

#### Métrica de desempeño del modelo

Queremos validar la métrica de desempeño del modelo. Normalmente, queremos comparar esta métrica obtenida para el modelo Challenger contra la del modelo Champion. Como aún no hemos registrado un modelo Champion, solo recuperaremos la métrica para el modelo Challenger sin hacer una comparación.

El modelo registrado captura información sobre la ejecución del experimento MLflow, donde las métricas del modelo se registraron durante el entrenamiento. Esto le brinda trazabilidad desde el modelo implementado hasta las ejecuciones de entrenamiento iniciales.

Aquí, usaremos el puntaje F1 para el conjunto de datos de prueba reservado durante el entrenamiento.

In [0]:
model_run_id = model_details.run_id
f1_score = mlflow.get_run(model_run_id).data.metrics['val_f1_score']

try:
    #Compare the challenger f1 score to the existing champion if it exists
    champion_model = client.get_model_version_by_alias(model_name, "Champion")
    champion_f1 = mlflow.get_run(champion_model.run_id).data.metrics['val_f1_score']
    print(f'Champion f1 score: {champion_f1}. Challenger f1 score: {f1_score}.')
    metric_f1_passed = f1_score >= champion_f1
except:
    print(f"No Champion found. Accept the model as it's the first one.")
    metric_f1_passed = True

print(f'Model {model_name} version {model_details.version} metric_f1_passed: {metric_f1_passed}')
# Tag that F1 metric check has passed
client.set_model_version_tag(name=model_name, version=model_details.version, key="metric_f1_passed", value=metric_f1_passed)

### Métricas de referencia o de negocio en el conjunto de datos de evaluación

Vamos a usar nuestro conjunto de datos de validación para verificar el impacto potencial del nuevo modelo.

***Nota: Esto es solo para evaluar nuestros modelos, no debe confundirse con la prueba A/B**. La prueba A/B se realiza en línea, dividiendo el tráfico entre 2 modelos. Requiere un ciclo de retroalimentación para evaluar el efecto de la predicción (por ejemplo, después de una predicción, ¿el descuento que ofrecimos al cliente evitó el churn?). Cubriremos la prueba A/B en la parte avanzada.*

In [0]:
import pyspark.sql.functions as F


# Get the eval dataset:
eval_df = spark.table('mlops_churn_training').filter("split='test'")

# Call the model with the given alias and return the prediction
def predict_churn(df, model_alias):
    model = mlflow.pyfunc.spark_udf(spark, model_uri=f"models:/{catalog}.{db}.mlops_churn@{model_alias}") #Use env_manager="virtualenv" to recreate a venv with the same python version if needed
    return df.withColumn('predictions', model(*model.metadata.get_input_schema().input_names()))

In [0]:
import pandas as pd
import plotly.express as px
from sklearn.metrics import confusion_matrix


#Note: this is over-simplified and depends on your use-case, but the idea is to evaluate our model against business metrics
cost_of_customer_churn = 2000 #in dollar
cost_of_discount = 500 #in dollar

cost_true_negative = 0 #did not churn, we did not give him the discount
cost_false_negative = cost_of_customer_churn #did churn, we lost the customer
cost_true_positive = cost_of_customer_churn -cost_of_discount #We avoided churn with the discount
cost_false_positive = -cost_of_discount #doesn't churn, we gave the discount for free

def get_model_value_in_dollar(model_alias):
    # Convert preds_df to Pandas DataFrame
    model_predictions = predict_churn(eval_df, model_alias).toPandas()
    # Calculate the confusion matrix
    tn, fp, fn, tp = confusion_matrix(model_predictions['churn'], model_predictions['predictions']).ravel()
    return tn * cost_true_negative+ fp * cost_false_positive + fn * cost_false_negative + tp * cost_true_positive
#add an exception to catch non-existing model champion yet
is_champ_model_exist = True
try:
    client.get_model_version_by_alias(f"{catalog}.{db}.mlops_churn", "Champion")
    print("Model already registered as Champion")
except Exception as error:
    print("An error occurred:", type(error).__name__, "It means no champion model yet exist")
    is_champ_model_exist = False
if is_champ_model_exist:
    champion_potential_revenue_gain = get_model_value_in_dollar("Champion")
    challenger_potential_revenue_gain = get_model_value_in_dollar("Challenger")

try:
    #Compare the challenger f1 score to the existing champion if it exists
    champion_potential_revenue_gain = get_model_value_in_dollar("Champion")
except:
    print(f"No Champion found. Accept the model as it's the first one.")
    champion_potential_revenue_gain = 0
    
challenger_potential_revenue_gain = get_model_value_in_dollar("Challenger")

data = {'Model Alias': ['Challenger', 'Champion'],
        'Potential Revenue Gain': [challenger_potential_revenue_gain, champion_potential_revenue_gain]}

# Create a bar plot using plotly express
px.bar(data, x='Model Alias', y='Potential Revenue Gain', color='Model Alias',
    labels={'Potential Revenue Gain': 'Revenue Impacted'},
    title='Business Metrics - Revenue Impacted')

## Resultados de validación

¡Eso es todo! Hemos demostrado algunas comprobaciones simples en el modelo. Veamos los resultados de la validación.

In [0]:
results = client.get_model_version(model_name, model_version)
results.tags

## Promoviendo el Challenger a Champion

Cuando estemos satisfechos con los resultados del modelo __Challenger__, podemos promoverlo a Champion. Esto se hace estableciendo su alias como `@Champion`. Las canalizaciones de inferencia que cargan el modelo usando el alias `@Champion` cargarán entonces este nuevo modelo. El alias en el modelo Champion anterior, si existe, se eliminará automáticamente. El modelo mantiene su alias `@Challenger` hasta que se implemente un nuevo modelo Challenger con el alias para reemplazarlo.

In [0]:
if results.tags["has_description"] == "True" and results.tags["metric_f1_passed"] == "True":
  print('register model as Champion!')
  client.set_registered_model_alias(
    name=model_name,
    alias="Champion",
    version=model_version
  )
else:
  raise Exception("Model not ready for promotion")

### ¡Felicidades! Nuestro modelo ha sido validado y promovido correctamente

Ahora tenemos la certeza de que nuestro modelo está listo para ser utilizado en canalizaciones de inferencia y endpoints de servicio en tiempo real, ya que cumple con nuestros estándares de validación.

Siguiente: [Ejecutar inferencia por lotes desde nuestro nuevo modelo Champion promovido]($./05_batch_inference)